In [61]:
import pandas as pd 


df = pd.read_csv('training/final_dataset.csv')

# Remove few column
dataset2 = df.copy().drop(columns =['Date', 'HomeTeam', 'AwayTeam', 'FTHG', 'FTAG',
       'HTGS', 'ATGS', 'HTGC', 'ATGC',
       'HM4', 'HM5','AM4', 'AM5', 'MW', 'HTFormPtsStr',
       'ATFormPtsStr', 'HTFormPts', 'ATFormPts', 'HTWinStreak3',
       'HTWinStreak5', 'HTLossStreak3', 'HTLossStreak5', 'ATWinStreak3',
       'ATWinStreak5', 'ATLossStreak3', 'ATLossStreak5',
       'DiffPts'] )

In [62]:
dataset2

,Unnamed: 0,FTR,HTP,ATP,HM1,HM2,HM3,AM1,AM2,AM3,HTGD,ATGD,DiffFormPts
0,0,H,0.000000,0.000000,M,M,M,M,M,M,0.000000,0.000000,0.000000
1,1,H,0.000000,0.000000,M,M,M,M,M,M,0.000000,0.000000,0.000000
2,2,NH,0.000000,0.000000,M,M,M,M,M,M,0.000000,0.000000,0.000000
3,3,NH,0.000000,0.000000,M,M,M,M,M,M,0.000000,0.000000,0.000000
4,4,H,0.000000,0.000000,M,M,M,M,M,M,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6835,6835,H,1.078947,1.842105,L,L,L,D,W,W,-0.289474,0.710526,-0.263158
6836,6836,NH,0.947368,2.552632,W,D,W,W,D,W,-0.473684,2.052632,-0.131579
6837,6837,NH,0.868421,0.789474,L,L,L,L,D,D,-0.710526,-0.894737,-0.052632
6838,6838,H,1.947368,1.236842,W,L,W,W,L,L,0.973684,-0.078947,0.078947


In [63]:
#what is the win rate for the home team?

# Total number of matches.
n_matches = dataset2.shape[0]

# Calculate number of features. -1 because we are saving one as the target variable (win/lose/draw)
n_features = dataset2.shape[1] - 1

# Calculate matches won by home team.
n_homewins = len(dataset2[dataset2.FTR == 'H'])

# Calculate win rate for home team.
win_rate = (float(n_homewins) / (n_matches)) * 100

# Print the results
print("Total number of matches: {}".format(n_matches))
print ("Number of features: {}".format(n_features))
print( "Number of matches won by home team: {}".format(n_homewins))
print ("Win rate of home team: {:.2f}%".format(win_rate))

Total number of matches: 6840
Number of features: 12
Number of matches won by home team: 3176
Win rate of home team: 46.43%


In [64]:
# Separate into feature set and target variable
#FTR = Full Time Result (H=Home Win, D=Draw, A=Away Win)
X_all = dataset2.drop(['FTR'],1)
y_all = dataset2['FTR']

# Standardising the data.
from sklearn.preprocessing import scale

#Center to the mean and component wise scale to unit variance.
cols = [['HTGD','ATGD','HTP','ATP']]
for col in cols:
    X_all[col] = scale(X_all[col])

#last 3 wins for both sides
X_all.HM1 = X_all.HM1.astype('str')
X_all.HM2 = X_all.HM2.astype('str')
X_all.HM3 = X_all.HM3.astype('str')
X_all.AM1 = X_all.AM1.astype('str')
X_all.AM2 = X_all.AM2.astype('str')
X_all.AM3 = X_all.AM3.astype('str')

#we want continous vars that are integers for our input data, so lets remove any categorical vars
def preprocess_features(X):
    ''' Preprocesses the football data and converts catagorical variables into dummy variables. '''
    
    # Initialize new output DataFrame
    output = pd.DataFrame(index = X.index)

    # Investigate each feature column for the data
    for col, col_data in X.iteritems():

        # If data type is categorical, convert to dummy variables
        if col_data.dtype == object:
            col_data = pd.get_dummies(col_data, prefix = col)
                    
        # Collect the revised columns
        output = output.join(col_data)
    
    return output

X_all = preprocess_features(X_all)
print ("Processed feature columns ({} total features):\n{}".format(len(X_all.columns), list(X_all.columns)))

Processed feature columns (30 total features):
['Unnamed: 0', 'HTP', 'ATP', 'HM1_D', 'HM1_L', 'HM1_M', 'HM1_W', 'HM2_D', 'HM2_L', 'HM2_M', 'HM2_W', 'HM3_D', 'HM3_L', 'HM3_M', 'HM3_W', 'AM1_D', 'AM1_L', 'AM1_M', 'AM1_W', 'AM2_D', 'AM2_L', 'AM2_M', 'AM2_W', 'AM3_D', 'AM3_L', 'AM3_M', 'AM3_W', 'HTGD', 'ATGD', 'DiffFormPts']


/var/folders/rd/wcpp_f4d5tn_q3_cpnymjrqh0000gq/T/ipykernel_38222/3391985158.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  X_all = dataset2.drop(['FTR'],1)
/var/folders/rd/wcpp_f4d5tn_q3_cpnymjrqh0000gq/T/ipykernel_38222/3391985158.py:30: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for col, col_data in X.iteritems():


In [65]:
X_all.head(10)

,Unnamed: 0,HTP,ATP,HM1_D,HM1_L,HM1_M,HM1_W,HM2_D,HM2_L,HM2_M,...,AM2_L,AM2_M,AM2_W,AM3_D,AM3_L,AM3_M,AM3_W,HTGD,ATGD,DiffFormPts
0,0,-2.280523,-2.345017,0,0,1,0,0,0,1,...,0,1,0,0,0,1,0,0.013972,-0.020983,0.0
1,1,-2.280523,-2.345017,0,0,1,0,0,0,1,...,0,1,0,0,0,1,0,0.013972,-0.020983,0.0
2,2,-2.280523,-2.345017,0,0,1,0,0,0,1,...,0,1,0,0,0,1,0,0.013972,-0.020983,0.0
3,3,-2.280523,-2.345017,0,0,1,0,0,0,1,...,0,1,0,0,0,1,0,0.013972,-0.020983,0.0
4,4,-2.280523,-2.345017,0,0,1,0,0,0,1,...,0,1,0,0,0,1,0,0.013972,-0.020983,0.0
5,5,-2.280523,-2.345017,0,0,1,0,0,0,1,...,0,1,0,0,0,1,0,0.013972,-0.020983,0.0
6,6,-2.280523,-2.345017,0,0,1,0,0,0,1,...,0,1,0,0,0,1,0,0.013972,-0.020983,0.0
7,7,-2.280523,-2.345017,0,0,1,0,0,0,1,...,0,1,0,0,0,1,0,0.013972,-0.020983,0.0
8,8,-2.280523,-2.345017,0,0,1,0,0,0,1,...,0,1,0,0,0,1,0,0.013972,-0.020983,0.0
9,9,-2.280523,-2.345017,0,0,1,0,0,0,1,...,0,1,0,0,0,1,0,0.013972,-0.020983,0.0


In [66]:
from sklearn.model_selection import train_test_split

# Shuffle and split the dataset into training and testing set.
X_train, X_test, y_train, y_test = train_test_split(X_all, y_all, 
                                                    test_size = 0.3,
                                                    random_state = 2,
                                                    stratify = y_all)

In [67]:
#fitting the RANDOM FOREST to the training se
from sklearn.ensemble import RandomForestClassifier
#classifier = RandomForestClassifier(n_estimators = 10, criterion = 'entropy', random_state = 0)
classifier = RandomForestClassifier(criterion='gini', 
                             n_estimators=700,
                             min_samples_split=10,
                             min_samples_leaf=1,
                             max_features='auto',
                             oob_score=True,
                             random_state=1,
                             n_jobs=-1)
classifier.fit(X_train, y_train)

/Users/tudor/.pyenv/versions/3.9.12/lib/python3.9/site-packages/sklearn/ensemble/_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(


RandomForestClassifier(max_features='auto', min_samples_split=10,
                       n_estimators=700, n_jobs=-1, oob_score=True,
                       random_state=1)

In [68]:
#predicting result
Y_pred = classifier.predict(X_test)

In [69]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, Y_pred)

In [70]:
y_test 

5216     H
3155    NH
4835    NH
2496     H
3502     H
        ..
5909    NH
3112    NH
2959    NH
3       NH
5729    NH
Name: FTR, Length: 2052, dtype: object

In [71]:
Y_pred

array(['H', 'H', 'NH', ..., 'NH', 'H', 'NH'], dtype=object)

In [72]:
true_positive = cm[0][0]
false_positive = cm[1][0]

true_negative = cm[1][1]
false_negative = cm[0][1]

accuracy = (true_positive + true_negative) / (true_positive + true_negative + false_positive + false_negative)
print("Accuracy: ", accuracy)

Accuracy:  0.6384015594541911


In [73]:
df

,Unnamed: 0,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTGS,ATGS,HTGC,...,HTLossStreak3,HTLossStreak5,ATWinStreak3,ATWinStreak5,ATLossStreak3,ATLossStreak5,HTGD,ATGD,DiffPts,DiffFormPts
0,0,19/08/00,Charlton,Man City,4,0,H,0,0,0,...,0,0,0,0,0,0,0.000000,0.000000,0.000000,0.000000
1,1,19/08/00,Chelsea,West Ham,4,2,H,0,0,0,...,0,0,0,0,0,0,0.000000,0.000000,0.000000,0.000000
2,2,19/08/00,Coventry,Middlesbrough,1,3,NH,0,0,0,...,0,0,0,0,0,0,0.000000,0.000000,0.000000,0.000000
3,3,19/08/00,Derby,Southampton,2,2,NH,0,0,0,...,0,0,0,0,0,0,0.000000,0.000000,0.000000,0.000000
4,4,19/08/00,Leeds,Everton,2,0,H,0,0,0,...,0,0,0,0,0,0,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6835,6835,13/05/18,Newcastle,Chelsea,3,0,H,36,62,47,...,0,0,1,0,0,0,-0.289474,0.710526,-0.763158,-0.263158
6836,6836,13/05/18,Southampton,Man City,0,1,NH,37,105,55,...,0,0,1,0,0,0,-0.473684,2.052632,-1.605263,-0.131579
6837,6837,13/05/18,Swansea,Stoke,1,2,NH,27,33,54,...,0,0,0,0,0,0,-0.710526,-0.894737,0.078947,-0.052632
6838,6838,13/05/18,Tottenham,Leicester,5,4,H,69,52,32,...,0,0,0,0,0,0,0.973684,-0.078947,0.710526,0.078947


In [74]:
y_pred_df = pd.DataFrame(Y_pred, columns=['Prediction'])
y_pred_df['index'] = range(0, len(y_pred_df))

X_test['index'] = range(0, len(X_test))

In [75]:
results = X_test.join(pd.DataFrame(Y_pred), how='inner', on='index')

In [76]:
results['Predictions'] = results[0]
results = results.drop(columns=[0])

In [77]:
results

,Unnamed: 0,HTP,ATP,HM1_D,HM1_L,HM1_M,HM1_W,HM2_D,HM2_L,HM2_M,...,AM2_W,AM3_D,AM3_L,AM3_M,AM3_W,HTGD,ATGD,DiffFormPts,index,Predictions
5216,5216,-0.798456,-0.911362,0,1,0,0,1,0,0,...,0,1,0,0,0,-1.479397,-1.670398,0.071429,0,H
3155,3155,0.863256,-0.114887,0,1,0,0,0,0,0,...,1,1,0,0,0,0.614752,-0.622332,-0.083333,1,H
4835,4835,-0.326889,-0.160400,0,1,0,0,0,1,0,...,0,0,1,0,0,-0.346497,-0.278704,-0.107143,2,NH
2496,2496,0.806096,-0.954806,0,1,0,0,0,0,0,...,0,0,1,0,0,1.193686,-1.267416,0.272727,3,H
3502,3502,0.444085,0.840884,1,0,0,0,0,0,0,...,1,0,0,0,1,0.494596,1.582614,-0.555556,4,NH
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5909,5909,0.953078,1.296012,1,0,0,0,0,0,0,...,0,0,1,0,0,1.249863,0.803724,0.095238,2047,NH
3112,3112,-0.630039,-0.433477,1,0,0,0,0,1,0,...,0,0,0,0,1,-0.346497,-0.381793,-0.125000,2048,NH
2959,2959,-0.079878,1.732936,0,1,0,0,0,0,0,...,1,0,0,0,1,0.350409,1.470362,-0.133333,2049,NH
3,3,-2.280523,-2.345017,0,0,1,0,0,0,1,...,0,0,0,1,0,0.013972,-0.020983,0.000000,2050,H


In [78]:
results = results.join(df, how='inner', on='Unnamed: 0', lsuffix='_caller', rsuffix='_other')
results[['Unnamed: 0', 'HomeTeam', 'AwayTeam', 'FTR', 'Predictions']]

,Unnamed: 0,HomeTeam,AwayTeam,FTR,Predictions
5216,5216,Cardiff,Fulham,H,H
3155,3155,Aston Villa,Middlesbrough,NH,H
4835,4835,Sunderland,Fulham,NH,NH
2496,2496,Arsenal,Charlton,H,H
3502,3502,Sunderland,Liverpool,H,NH
...,...,...,...,...,...
5909,5909,Tottenham,Leicester,NH,NH
3112,3112,Fulham,Sunderland,NH,NH
2959,2959,Tottenham,Chelsea,NH,NH
3,3,Derby,Southampton,NH,H
